In [ ]:
from pybis import Openbis
import json
import ipywidgets as widgets
from IPython.display import display, Javascript
from datetime import datetime

In [ ]:
# Global variables
OPENBIS_SESSION = None
RAW_MATERIALS_TYPES = ["CRYSTAL", "WAFER_SUBSTRATE", "2D_LAYERED_MATERIAL", "MOLECULE"]
PROCESS_SAMPLE_TYPES = ["SPUTTERING", "ANNEALING", "DEPOSITION"]

In [ ]:
# Widgets
samples_dropdown = widgets.Dropdown(description='Sample', disabled=False, layout = widgets.Layout(width = '400px'))
samples_dropdown.style = {'description_width': '110px'}
sample_details_textbox = widgets.Textarea(value = '', description = '', disabled = True, layout = widgets.Layout(width = '600px', height = '300px'))
sample_metadata_boxes = widgets.HBox([samples_dropdown, sample_details_textbox])

instruments_dropdown = widgets.Dropdown(description='Select instrument', disabled=False, layout = widgets.Layout(width = '993px'))
instruments_dropdown.style = {'description_width': '110px'}

experiments_dropdown = widgets.Dropdown(description='Select experiment', disabled=False, layout = widgets.Layout(width = '993px'))
experiments_dropdown.style = {'description_width': '110px'}

duration_value_floatbox = widgets.FloatText(value = 0, description = 'Duration', disabled = False, layout = widgets.Layout(width = '200px'))
duration_value_floatbox.style = {'description_width': '110px'}
duration_unit_dropdown = widgets.Dropdown(value = 'sec', description = '', disabled = False, options = ["sec", "min", "hrs"], layout = widgets.Layout(width = '100px'))
duration_hbox = widgets.HBox([duration_value_floatbox, duration_unit_dropdown])

pressure_value_floatbox = widgets.FloatText(value = 0, description = 'Pressure', disabled = False, layout = widgets.Layout(width = '200px'))
pressure_value_floatbox.style = {'description_width': '110px'}
pressure_unit_dropdown = widgets.Dropdown(value = 'mBar', description = '', disabled = False, options = ["mBar", "Bar", "Pa", "kPa"], layout = widgets.Layout(width = '100px'))
pressure_hbox = widgets.HBox([pressure_value_floatbox, pressure_unit_dropdown])

discharge_voltage_value_floatbox = widgets.FloatText(value = 0, description = 'Discharge voltage', disabled = False, layout = widgets.Layout(width = '200px'))
discharge_voltage_value_floatbox.style = {'description_width': '110px'}
discharge_voltage_unit_dropdown = widgets.Dropdown(value = 'kV', description = '', disabled = False, options = ["kV"], layout = widgets.Layout(width = '100px'))
discharge_voltage_hbox = widgets.HBox([discharge_voltage_value_floatbox, discharge_voltage_unit_dropdown])

voltage_value_floatbox = widgets.FloatText(value = 0, description = 'Voltage', disabled = False, layout = widgets.Layout(width = '200px'))
voltage_value_floatbox.style = {'description_width': '110px'}
voltage_unit_dropdown = widgets.Dropdown(value = 'V', description = '', disabled = False, options = ["V", "mV", "uV"], layout = widgets.Layout(width = '100px'))
voltage_hbox = widgets.HBox([voltage_value_floatbox, voltage_unit_dropdown])

properties_on_left = widgets.VBox([duration_hbox, pressure_hbox, discharge_voltage_hbox, voltage_hbox])

temperature_value_floatbox = widgets.FloatText(value = 0, description = 'Temperature', disabled = False, layout = widgets.Layout(width = '200px'))
temperature_value_floatbox.style = {'description_width': '110px'}
temperature_unit_dropdown = widgets.Dropdown(value = 'K', description = '', disabled = False, options = ["K", "oC", "oF"], layout = widgets.Layout(width = '100px'))
temperature_hbox = widgets.HBox([temperature_value_floatbox, temperature_unit_dropdown])

angle_value_floatbox = widgets.FloatText(value = 0, description = 'Angle', disabled = False, layout = widgets.Layout(width = '200px'))
angle_value_floatbox.style = {'description_width': '110px'}
angle_unit_dropdown = widgets.Dropdown(value = 'deg', description = '', disabled = False, options = ["deg", "rad"], layout = widgets.Layout(width = '100px'))
angle_hbox = widgets.HBox([angle_value_floatbox, angle_unit_dropdown])

current_value_floatbox = widgets.FloatText(value = 0, description = 'Current', disabled = False, layout = widgets.Layout(width = '200px'))
current_value_floatbox.style = {'description_width': '110px'}
current_unit_dropdown = widgets.Dropdown(value = 'A', description = '', disabled = False, options = ["A", "mA", "uA"], layout = widgets.Layout(width = '100px'))
current_hbox = widgets.HBox([current_value_floatbox, current_unit_dropdown])

properties_on_right = widgets.VBox([temperature_hbox, angle_hbox, current_hbox])

sputter_properties = widgets.HBox([properties_on_left, properties_on_right])

comments_textbox = widgets.Textarea(value = '', placeholder = "Write comments here...", description = 'Comments', disabled = False, layout = widgets.Layout(width = '993px', height = '200px'))
comments_textbox.style = {'description_width': '110px'}

sputtering_name_textbox = widgets.Text(value = '', placeholder = "Write name here...", description = 'Name', disabled = False, layout = widgets.Layout(width = '400px'))
sputtering_name_textbox.style = {'description_width': '110px'}

sample_out_name_textbox = widgets.Text(value = '', placeholder = "Write output sample name here...", description = 'Name', disabled = False, layout = widgets.Layout(width = '400px'))
sample_out_name_textbox.style = {'description_width': '110px'}

create_sputtering_button = widgets.Button(description = 'Save', disabled = False, button_style = '', tooltip = 'Save sputtering information')
quit_button = widgets.Button(description = 'Go back', disabled = False, button_style = '', tooltip = 'Go back')

bottom_buttons_hbox = widgets.HBox([create_sputtering_button, quit_button])

In [ ]:
# Functions
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Function to close the notebook page
def close_notebook(b):
    display(Javascript(f'window.open("home.ipynb", "_blank")'))
    display(Javascript('window.close()'))

def create_sputtering_action(b):
    global OPENBIS_SESSION
    if experiments_dropdown.value == "-1":
        print("Select an experiment.")
    elif samples_dropdown.value == "-1":
        print("Select a sample.")
    elif instruments_dropdown.value == "-1":
        print("Select an instrument.")
    else:
        sample_parents = [samples_dropdown.value, instruments_dropdown.value]
        object_properties = {}
        object_properties["$name"] = sputtering_name_textbox.value
        object_properties["duration"] = json.dumps({"value": duration_value_floatbox.value, "unit": duration_unit_dropdown.value})
        object_properties["pressure"] = json.dumps({"value": pressure_value_floatbox.value, "unit": pressure_unit_dropdown.value})
        object_properties["discharge_voltage"] = json.dumps({"value": discharge_voltage_value_floatbox.value, "unit": discharge_voltage_unit_dropdown.value})
        object_properties["voltage"] = json.dumps({"value": voltage_value_floatbox.value, "unit": voltage_unit_dropdown.value})
        object_properties["temperature"] = json.dumps({"value": temperature_value_floatbox.value, "unit": temperature_unit_dropdown.value})
        object_properties["angle"] = json.dumps({"value": angle_value_floatbox.value, "unit": angle_unit_dropdown.value})
        object_properties["current"] = json.dumps({"value": current_value_floatbox.value, "unit": current_unit_dropdown.value})
        object_properties["comments"] = comments_textbox.value
        
        sputter_task = OPENBIS_SESSION.new_object(type = "SPUTTERING", 
                                                collection = experiments_dropdown.value,
                                                props = object_properties,
                                                parents = sample_parents)
        sputter_task.save()

        OPENBIS_SESSION.new_object(type = "SAMPLE",
                                   collection = "20240716082516377-60",
                                   props = {"$name": sample_out_name_textbox.value},
                                   parents = [sputter_task]).save()

def load_sample_metadata(change):
    global OPENBIS_SESSION
    if samples_dropdown.value == "-1":
        sample_details_textbox.value = ''
    else:
        sample_object = OPENBIS_SESSION.get_object(samples_dropdown.value)
        sample_metadata = sample_object.props.all()
        sample_parents_metadata = []
        sample_parents_metadata = get_parents_recursive(sample_object, sample_parents_metadata)
        
        sample_processes_strings = []
        sample_materials_strings = []
        number_parents = len(sample_parents_metadata)
        parent_idx = 0
        while parent_idx < number_parents:
            parent_metadata = sample_parents_metadata[parent_idx]
            if parent_metadata[0] == "DEPOSITION":
                next_parent_metadata = sample_parents_metadata[parent_idx + 1]
                sample_metadata_string = f"> {parent_metadata[0]} ({parent_metadata[3]}, {parent_metadata[1]}, {parent_metadata[2]}) [{next_parent_metadata[0]} ({next_parent_metadata[3]}, {next_parent_metadata[1]}, {next_parent_metadata[4]})]"
                parent_idx += 1
            else:
                sample_metadata_string = f"> {parent_metadata[0]} ({parent_metadata[3]}, {parent_metadata[1]}, {parent_metadata[2]})"
            
            if parent_metadata[0] in PROCESS_SAMPLE_TYPES:
                sample_processes_strings.append(sample_metadata_string)
            else:
                sample_materials_strings.append(sample_metadata_string)
                
            parent_idx += 1
        
        sample_metadata_string = f"PermId: {sample_object.attrs.permId}\nMaterial:"
        
        for material_string in sample_materials_strings:
            sample_metadata_string = f"{sample_metadata_string}\n{material_string}"
            
        sample_metadata_string = f"{sample_metadata_string}\nProcesses:"
        for process_string in sample_processes_strings:
            sample_metadata_string = f"{sample_metadata_string}\n{process_string}"
        
        sample_details_textbox.value = sample_metadata_string

def get_parents_recursive(object, object_parents_metadata):
    if object.attrs.type in PROCESS_SAMPLE_TYPES or object.attrs.type in RAW_MATERIALS_TYPES:
        object_parents_metadata.append([object.attrs.type, object.attrs.permId, object.attrs.registrationDate, 
                                        object.props['$name'], object.props['sum_formula']])
    object_parents = object.parents
    for parent in object_parents:
        parent_object = OPENBIS_SESSION.get_object(parent)
        get_parents_recursive(parent_object, object_parents_metadata)
    return object_parents_metadata

In [ ]:
# Connect with openBIS
session_data = read_json("token.json")
OPENBIS_SESSION = Openbis(session_data["url"], verify_certificates = False)
OPENBIS_SESSION.set_token(session_data["token"])

# Sputtering

### Select experiment, sample, and instrument

In [ ]:
experiments = OPENBIS_SESSION.get_collections(type = "EXPERIMENT")
samples = OPENBIS_SESSION.get_objects(type = "SAMPLE")
instruments = OPENBIS_SESSION.get_objects(type = "INSTRUMENT")

experiments_names_permids = [(f"{experiment.props['$name']} ({experiment.attrs.identifier})", experiment.permId) for experiment in experiments]
experiments_names_permids.insert(0, ('Select an experiment...', "-1"))
experiments_dropdown.options = experiments_names_permids

samples_names_permids = [(sample.props['$name'], sample.permId) for sample in samples]
samples_dates = [datetime.fromisoformat(sample.registrationDate) for sample in samples]
samples_names_permids = [x for _,x in sorted(zip(samples_dates, samples_names_permids), reverse = True)]
samples_names_permids.insert(0, ('Select an input sample...', "-1"))
samples_dropdown.options = samples_names_permids

instruments_names_permids = [(f"{instrument.props['$name']} ({instrument.attrs.permId})", instrument.permId) for instrument in instruments]
instruments_names_permids.insert(0, ('Select an instrument...', "-1"))
instruments_dropdown.options = instruments_names_permids
instruments_dropdown.options = instruments_names_permids

display(widgets.VBox([experiments_dropdown, sample_metadata_boxes, instruments_dropdown]))
create_sputtering_button.on_click(create_sputtering_action)
quit_button.on_click(close_notebook)
samples_dropdown.observe(load_sample_metadata, names = 'value')

### Properties

In [ ]:
display(sputtering_name_textbox, sputter_properties, comments_textbox)

### Sample out

In [ ]:
display(sample_out_name_textbox)

### Save in openBIS

In [ ]:
display(bottom_buttons_hbox)